<a href="https://colab.research.google.com/github/jdannem6/ML_Forex_Forecasting/blob/main/path_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Conversion rates matrix (5 days x 6 currencies)
# format important
conversion_rates = np.array([
    [1.03, 0.9, 1.08, 0.75, 0.82, 1.23],  # DAY 1: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
    [1.04, 0.93, 1.03, 0.65, 0.82, 1.12],  # DAY 2: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
    [1.07, 0.95, 1.04, 0.55, 0.80, 1.01],  # DAY 3: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
    [1.05, 0.89, 1.06, 0.45, 0.65, 1.004],  # DAY 4: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
    [1.02, 0.93, 1.03, 0.33, 0.32, 1.003]  # DAY 5: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
])


In [ ]:
def viterbi_strategy(conversion_rates):

    dp = []
    path = []

    curr = ['USD', 'EUR', 'CHF', 'GBP']
    prev_day = np.array([1, 0, 0, 0])

    for i in range(5):

        USD, EUR, CHF, GBP = prev_day
        USD_EUR, EUR_CHF, EUR_GBP, GBP_CHF, USD_CHF, USD_GBP = conversion_rates[i, :]
        EUR_USD, CHF_EUR, GBP_EUR, CHF_GBP, CHF_USD, GBP_USD = 1/conversion_rates[i, :]

        if i<=3:
            dp = np.array([max(USD*1, EUR*EUR_USD, CHF*CHF_USD, GBP*GBP_USD), max(USD*USD_EUR, EUR*1, CHF*CHF_EUR, GBP*GBP_EUR), max(USD*USD_CHF, EUR*EUR_CHF, CHF*1, GBP*GBP_CHF), max(USD*USD_GBP, EUR*EUR_GBP, CHF*CHF_GBP, GBP*1)])
            path.append(np.array([curr[np.argmax([USD*1, EUR*EUR_USD, CHF*CHF_USD, GBP*GBP_USD])], curr[np.argmax([USD*USD_EUR, EUR*1, CHF*CHF_EUR, GBP*GBP_EUR])], curr[np.argmax([USD*USD_CHF, EUR*EUR_CHF, CHF*1, GBP*GBP_CHF])], curr[np.argmax([USD*USD_GBP, EUR*EUR_GBP, CHF*CHF_GBP, GBP*1])]]))
            prev_day = dp

        elif i==4:
            dp = np.array([USD*1, EUR*EUR_USD, CHF*CHF_USD, GBP*GBP_USD])
            path.append(curr)

    final_path = ['USD']
    index = curr.index(path[-1][np.argmax(dp)])
    print(1000*np.max(dp))

    for i in range(4, -1, -1):
        prev_currency = path[i][index]
        final_path.append(prev_currency)
        index = curr.index(prev_currency)

    final_path.reverse()
    final_path = [final_path[i] + '-' + final_path[i+1] for i in range(5)]

    return final_path



In [ ]:
viterbi_strategy(conversion_rates)

3545.2063106796113


['USD-GBP', 'GBP-EUR', 'EUR-CHF', 'CHF-CHF', 'CHF-USD']

In [ ]:
def viterbi_distributed_strategy(conversion_rates, currencies=['A', 'B'], amount=250):

    dp = []
    path = []

    curr = currencies
    prev_day = np.array([1, 0])

    for i in range(5):
        A, B = prev_day
        A_B = conversion_rates[i]
        B_A = 1/conversion_rates[i]

        if i<=3:
            dp = np.array([max(A*1, B*B_A), max(A*A_B, B*1)])
            path.append(np.array([curr[np.argmax([A*1, B*B_A])], curr[np.argmax([A*A_B, B*1])]]))
            prev_day = dp

        elif i==4:
            dp = np.array([A*1, B*B_A])
            path.append(curr)

    final_path = ['A']
    index = curr.index(path[-1][np.argmax(dp)])
    final_amount = amount*np.max(dp)

    for i in range(4, -1, -1):
        prev_currency = path[i][index]
        final_path.append(prev_currency)
        index = curr.index(prev_currency)

    final_path.reverse()
    final_path = [final_path[i] + '-' + final_path[i+1] for i in range(5)]

    return final_path, final_amount


In [ ]:
final_path_USD_EUR, final_amount_USD_EUR = viterbi_distributed_strategy(conversion_rates[:, 0], currencies=['USD', 'EUR'], amount=333.333)
final_path_USD_CHF, final_amount_USD_CHF = viterbi_distributed_strategy(conversion_rates[:, 4], currencies=['USD', 'CHF'], amount=333.333)
final_path_USD_GBP, final_amount_USD_GBP = viterbi_distributed_strategy(conversion_rates[:, 5], currencies=['USD', 'GBP'], amount=333.333)

print('final_path_USD_EUR', final_path_USD_EUR, final_amount_USD_EUR)
print('final_path_USD_CHF', final_path_USD_CHF, final_amount_USD_CHF)
print('final_path_USD_GBP', final_path_USD_GBP, final_amount_USD_GBP)
print()

print('final_amount', final_amount_USD_EUR+final_amount_USD_CHF+final_amount_USD_GBP)

final_path_USD_EUR ['USD-USD', 'USD-USD', 'USD-EUR', 'EUR-EUR', 'EUR-A'] 349.67285294117653
final_path_USD_CHF ['USD-USD', 'USD-CHF', 'CHF-CHF', 'CHF-CHF', 'CHF-A'] 854.1658125
final_path_USD_GBP ['USD-GBP', 'GBP-GBP', 'GBP-GBP', 'GBP-GBP', 'GBP-A'] 408.7732701894318

final_amount 1612.6119356306083


In [ ]:
from numpy import array


available_paths = {'USD': ['USD_USD', 'USD_EUR', 'USD_GBP', 'USD_CHF'], 'EUR': ['EUR_USD', 'EUR_EUR', 'EUR_GBP', 'EUR_CHF'], 'CHF': ['CHF_USD', 'CHF_EUR', 'CHF_GBP', 'CHF_CHF'], 'GBP':['GBP_USD', 'GBP_EUR', 'GBP_GBP', 'GBP_CHF']}

def beam_search(distances, beta=5):

    paths_so_far = [[['USD_USD'], 1]]

    for idx, tier in enumerate(distances[:-1]):

        USD_EUR, EUR_CHF, EUR_GBP, GBP_CHF, USD_CHF, USD_GBP = tier
        EUR_USD, CHF_EUR, GBP_EUR, CHF_GBP, CHF_USD, GBP_USD = 1/tier
        d = {'USD_EUR':USD_EUR, 'EUR_CHF':EUR_CHF, 'EUR_GBP':EUR_GBP, 'GBP_CHF':GBP_CHF, 'USD_CHF':USD_CHF, 'USD_GBP':USD_GBP, 'EUR_USD':EUR_USD, 'CHF_EUR':CHF_EUR, 'GBP_EUR':GBP_EUR, 'CHF_GBP':CHF_GBP, 'CHF_USD':CHF_USD, 'GBP_USD':GBP_USD, 'USD_USD':1, 'EUR_EUR':1, 'GBP_GBP':1, 'CHF_CHF':1}

        paths_at_tier = list()

        for i in range(len(paths_so_far)):
            path, distance = paths_so_far[i]

            for j in range(4):
                path_extended = [path + [available_paths[path[-1][4:]][j]], distance*d[available_paths[path[-1][4:]][j]]]
                paths_at_tier.append(path_extended)

        # print(paths_at_tier[0][-1][4:])
        # print(available_paths[paths_at_tier[0][-1][4:]][0])
        print(paths_at_tier)
        paths_ordered = sorted(paths_at_tier, reverse=True, key=lambda element: element[1]*d[available_paths[element[0][-1][4:]][0]])

        paths_so_far = paths_ordered[:beta]
        print(idx, paths_so_far)

    paths_final = [(path.append(available_paths[path[-1][4:]][0]), val*d[available_paths[path[-1][4:]][0]]) for path, val in paths_so_far]

    return paths_so_far


#Distance matrix
dists = np.array([
    [1.03, 0.9, 1.08, 0.75, 0.82, 1.23],  # DAY 1: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
    [1.04, 0.93, 1.03, 0.65, 0.82, 1.12],  # DAY 2: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
    [1.07, 0.95, 1.04, 0.55, 0.80, 1.01],  # DAY 3: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
    [1.05, 0.89, 1.06, 0.45, 0.65, 1.004],  # DAY 4: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
    [1.02, 0.93, 1.03, 0.33, 0.32, 1.003]  # DAY 5: USD-EUR, EUR-CHF, EUR-GBP, GBP_CHF, USD-CHF, USD-GBP
])

# Calculating the best paths
best_paths = beam_search(dists, 5)
print('\nThe best paths:')
for beta_path in best_paths:
    print(beta_path)


[[['USD_USD', 'USD_USD'], 1], [['USD_USD', 'USD_EUR'], 1.03], [['USD_USD', 'USD_GBP'], 1.23], [['USD_USD', 'USD_CHF'], 0.82]]
0 [[['USD_USD', 'USD_USD'], 1], [['USD_USD', 'USD_EUR'], 1.03], [['USD_USD', 'USD_GBP'], 1.23], [['USD_USD', 'USD_CHF'], 0.82]]
[[['USD_USD', 'USD_USD', 'USD_USD'], 1], [['USD_USD', 'USD_USD', 'USD_EUR'], 1.04], [['USD_USD', 'USD_USD', 'USD_GBP'], 1.12], [['USD_USD', 'USD_USD', 'USD_CHF'], 0.82], [['USD_USD', 'USD_EUR', 'EUR_USD'], 0.9903846153846153], [['USD_USD', 'USD_EUR', 'EUR_EUR'], 1.03], [['USD_USD', 'USD_EUR', 'EUR_GBP'], 1.0609], [['USD_USD', 'USD_EUR', 'EUR_CHF'], 0.9579000000000001], [['USD_USD', 'USD_GBP', 'GBP_USD'], 1.0982142857142856], [['USD_USD', 'USD_GBP', 'GBP_EUR'], 1.1941747572815533], [['USD_USD', 'USD_GBP', 'GBP_GBP'], 1.23], [['USD_USD', 'USD_GBP', 'GBP_CHF'], 0.7995], [['USD_USD', 'USD_CHF', 'CHF_USD'], 0.9999999999999999], [['USD_USD', 'USD_CHF', 'CHF_EUR'], 0.8817204301075268], [['USD_USD', 'USD_CHF', 'CHF_GBP'], 1.2615384615384613], [